References

MABe Nearest Neighbors: The Original ⭐️⭐️⭐️⭐️⭐️
MABe EDA which makes sense ⭐️⭐️⭐️⭐️⭐️
MABe Validated baseline without machine learning
Squeeze GBT
Social Action Recognition in Mice | XGBoost1


# Imports and configs

In [1]:
"""F Beta customized for the data format of the MABe challenge."""

import json

from collections import defaultdict

import pandas as pd
import polars as pl


class HostVisibleError(Exception):
    pass


def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()  # ty: ignore
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            # Since the labels are sparse, we can't evaluate prediction keys not in the active labels.
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue

            new_frames = set(range(row['start_frame'], row['stop_frame']))
            # Ignore truly redundant predictions.
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                # A single agent can have multiple targets per frame (ex: evading all other mice) but only one action per target per frame.
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)


def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    # Need to align based on video IDs as we can't rely on the row IDs for handling public/private splits.
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    """
    F1 score for the MABe Challenge
    """
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

In [2]:
!pip install /kaggle/input/koolbox-library/koolbox-0.1.3-py3-none-any.whl --no-deps


Processing /kaggle/input/koolbox-library/koolbox-0.1.3-py3-none-any.whl


In [3]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score
from sklearn.base import clone
from xgboost import XGBClassifier
from tqdm.notebook import tqdm
from koolbox import Trainer
import numpy as np
import itertools
import warnings
import optuna
import joblib
import glob
import gc
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import os
from sklearn.pipeline import make_pipeline
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from typing import Dict, Optional, Tuple
from scipy.signal import savgol_filter
from sklearn.model_selection import StratifiedShuffleSplit



optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')
SEED = 1234
print('ready')

ready


## Chiến thuật "Lấy mẫu phân tầng" (Stratified Subsampling)

In [4]:
class StratifiedSubsetClassifierWEval(ClassifierMixin, BaseEstimator):
    def __init__(self,
                 estimator,
                 n_samples=None,
                 random_state: int = 42,
                 valid_size: float = 0.10,
                 val_cap_ratio: float = 0.25,
                 es_rounds: "int|str" = "auto",
                 es_metric: str = "auto"):
        self.estimator = estimator
        self.n_samples = (int(n_samples) if (n_samples is not None) else None)
        self.random_state = random_state
        self.valid_size = float(valid_size)
        self.val_cap_ratio = float(val_cap_ratio)
        self.es_rounds = es_rounds
        self.es_metric = es_metric
 
    def fit(self, X: pd.DataFrame, y):
        y = np.asarray(y)
        n_total = len(y); assert n_total == len(X)
        pos_rate = float(np.mean(y == 1))
    
        # if 1% positive example
        if pos_rate < 0.01:  # < 1% positive samples
            print(f"Rare class: {pos_rate*100:.2f}% positive")
            
            # double the requested sample size
            if self.n_samples is not None:
                self.n_samples = min(self.n_samples * 2, n_total)
            
            # for XGBoost: class weighting
            if self._is_xgb(self.estimator):
                n_pos = max(1, int((y == 1).sum()))
                n_neg = max(1, len(y) - n_pos)
                # triple scaled_pos_weight to penalize missing rare events
                self.estimator.set_params(scale_pos_weight=3.0 * (n_neg / n_pos))
                
        # train and validate indices
        tr_idx, va_idx = self._compute_train_val_indices(y, n_total)
        # just subset the data
        Xtr = X.iloc[tr_idx]; ytr = y[tr_idx]
        Xtr = Xtr.to_numpy(np.float32, copy=False)

        Xva = yva = None
        if va_idx is not None and len(va_idx) > 0:
            Xva = X.iloc[va_idx].to_numpy(np.float32, copy=False); yva = y[va_idx]

        # calculate positive rate in validation
        pos_rate = None
        if yva is not None and len(yva) > 0:
            pos_rate = float(np.mean(yva == 1))

        # Decide metric & patience
        metric = self._choose_metric(pos_rate)
        patience = self._choose_patience(pos_rate)

        # Apply imbalance knobs per library
        if self._is_xgb(self.estimator):
            # scale_pos_weight = n_neg / n_pos on TRAIN
            n_pos = max(1, int((ytr == 1).sum()))
            n_neg = max(1, len(ytr) - n_pos)
            self.estimator.set_params(scale_pos_weight=(n_neg / n_pos))
            self.estimator.set_params(eval_metric=metric)

        elif self._is_catboost(self.estimator):
            # GPU-safe auto balancing
            try: self.estimator.set_params(auto_class_weights="Balanced")
            except Exception: pass
            try: self.estimator.set_params(eval_metric=metric)
            except Exception: pass

        # Fit with ES if we have any validation (single-class OK with Logloss)
        has_valid = (Xva is not None and len(yva) > 0)
        if has_valid and self._is_xgb(self.estimator):
            import xgboost as xgb
            self.estimator.fit(
                Xtr, ytr,
                eval_set=[(Xva, yva)],
                verbose=False,
                callbacks=[xgb.callback.EarlyStopping(
                    rounds=int(patience),
                    metric_name=metric,
                    data_name="validation_0",
                    save_best=True
                )]
            )
        elif has_valid and self._is_catboost(self.estimator):
            from catboost import Pool
            self.estimator.set_params(
                use_best_model=True,
                od_type="Iter",
                od_wait=int(patience),
                custom_metric=["PRAUC:type=Classic;hints=skip_train~true"],
            )
            self.estimator.fit(
                Xtr, ytr,
                eval_set=Pool(Xva, yva),
                verbose=False,
                metric_period=50
            )
        else:
            # Fall back: train on train split without ES
            self.estimator.fit(Xtr, ytr)

        self.classes_ = getattr(self.estimator, "classes_", np.array([0, 1]))
        self._tr_idx_ = tr_idx; self._va_idx_ = va_idx; self._pos_rate_ = pos_rate
        return self

    def predict_proba(self, X: pd.DataFrame):
        return self.estimator.predict_proba(X)

    def predict(self, X: pd.DataFrame):
        return self.estimator.predict(X)

    # helpers
    def _compute_train_val_indices(self, y: np.ndarray, n_total: int):
        """
        creates stratified indices.
        if n_samples < n_total, subsamples the training data while keeping the validation set separate.
        """
        rng = np.random.default_rng(self.random_state)
        n_classes = np.unique(y).size

        # cannot stratify?? random split
        def full_data_split():
            if self.valid_size <= 0 or n_classes < 2:
                idx = rng.permutation(n_total); return idx, None
            sss = StratifiedShuffleSplit(n_splits=1, test_size=self.valid_size, random_state=self.random_state)
            tr, va = next(sss.split(np.zeros(n_total, dtype=np.int8), y))
            return tr, va
            
        # n_samples is the size limit, if no limit, use all
        if self.n_samples is None or self.n_samples >= n_total:
            return full_data_split()

        # sss = stratified shuffle split; select training subset
        sss_tr = StratifiedShuffleSplit(n_splits=1, train_size=self.n_samples, random_state=self.random_state)
        tr_idx, rest_idx = next(sss_tr.split(np.zeros(n_total, dtype=np.int8), y))
        remaining = len(rest_idx)
        
        # Select the VALIDATION subset from the 'rest' (remaining data)
        # We don't want validation data overlapping with training data.
        min_val_needed = int(np.ceil(self.n_samples * max(self.valid_size, 0.0)))
        val_cap = max(min_val_needed, int(round(self.val_cap_ratio * self.n_samples)))
        want_val = min(remaining, val_cap)

        y_rest = y[rest_idx]
        if remaining < min_val_needed or np.unique(y_rest).size < 2 or self.valid_size <= 0:
            return full_data_split()

        sss_val = StratifiedShuffleSplit(n_splits=1, train_size=want_val, random_state=self.random_state)
        try:
            va_sel, _ = next(sss_val.split(np.zeros(remaining, dtype=np.int8), y_rest))
        except ValueError:
            return full_data_split()

        va_idx = rest_idx[va_sel]
        return tr_idx, va_idx

    def _choose_metric(self, pos_rate=0.01) -> str:
        if self.es_metric != "auto":
            return self.es_metric
        if pos_rate is None or pos_rate == 0.0 or pos_rate == 1.0:
            return "logloss" if self._is_xgb(self.estimator) else "Logloss"
        return "aucpr" if self._is_xgb(self.estimator) else "PRAUC:type=Classic"

    def _choose_patience(self, pos_rate: Optional[float]) -> int:
        if isinstance(self.es_rounds, int):
            return self.es_rounds
        try:
            n_estimators = (int(self.estimator.get_params().get("n_estimators", 200))
                            if self._is_xgb(self.estimator)
                            else int(self.estimator.get_params().get("iterations", 500)))
        except Exception:
            n_estimators = 200
        base = max(30, int(round(0.20 * (n_estimators or 200))))
        if pos_rate is None:
            return base
        # Increase patience for highly imbalanced data
        if pos_rate < 0.005:   # <0.5%
            return int(round(base * 1.75))
        if pos_rate < 0.02:    # <2%
            return int(round(base * 1.40))
        return base

    # helpers to detect model
    @staticmethod
    def _is_xgb(est):
        name = est.__class__.__name__.lower(); mod = getattr(est, "__module__", "")
        return "xgb" in name or "xgboost" in mod or hasattr(est, "get_xgb_params")

    @staticmethod
    def _is_catboost(est):
        name = est.__class__.__name__.lower(); mod = getattr(est, "__module__", "")
        return "catboost" in name or "catboost" in mod or hasattr(est, "get_all_params")


In [5]:


class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    def __init__(self, estimator, n_samples, random_state=SEED):
        self.estimator = estimator
        self.n_samples = n_samples and int(n_samples)
        self.random_state = random_state

    def fit(self, X, y):
        """
         select a subset of data, then fit the base model.
        """
        y = np.asarray(y)
        n_total = len(y)

        # if no limit or less data than limit, just use all
        if self.n_samples is None or self.n_samples >= n_total:
            rng = np.random.default_rng(self.random_state)
            idx = rng.permutation(n_total)
        # or use a subset
        else:
            sss = StratifiedShuffleSplit(
                n_splits=1, train_size=self.n_samples, random_state=self.random_state
            )
            idx, _ = next(sss.split(np.zeros(n_total, dtype=np.int8), y))

        Xn = X.iloc[idx]
        Xn = Xn.to_numpy(np.float32, copy=False)
        yn = y[idx]

        # train for smaller subset
        self.estimator.fit(Xn, yn)
        self.classes_ = getattr(self.estimator, "classes_", np.array([0, 1]))
        return self

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def predict(self, X):
        return self.estimator.predict(X)


In [6]:
class CFG:
    # 1. Cấu hình đường dẫn (Giữ nguyên)
    train_path = "/kaggle/input/MABe-mouse-behavior-detection/train.csv"
    test_path = "/kaggle/input/MABe-mouse-behavior-detection/test.csv"
    train_annotation_path = "/kaggle/input/MABe-mouse-behavior-detection/train_annotation"
    train_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/train_tracking"
    test_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/test_tracking"

    model_path = "/kaggle/input/social-action-recognition-in-mice-xgb-catboost"
    model_name = "ensemble_v1" # Đổi tên để biết là dùng Ensemble

    # ====================================================
    # 2. CẤU HÌNH CHẾ ĐỘ CHẠY (QUAN TRỌNG)
    # ====================================================
    
    # Bật True: Chạy siêu nhanh trên ít dữ liệu để kiểm tra lỗi (Test code)
    # Bật False: Chạy thật để lấy kết quả (Train full hoặc Submit)
    debug = True  
    
    # --- ĐOẠN NÀY QUAN TRỌNG NHẤT ---
    # Kiểm tra: Nếu không phải đang ngồi code (Interactive) => Tức là đang Submit/Save Version
    # Thì CƯỠNG CHẾ tắt Debug ngay lập tức!
    if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') != 'Interactive':
        debug = False
        print("🚀 DETECTED SUBMISSION: Hệ thống tự động chuyển DEBUG = False để train full!")
    # --------------------------------
    
    if debug:
        print("🐞 DEBUG MODE: ON (Chạy test nhanh)")
        mode = 'validate'       # Luôn validate để xem code có crash không
        n_estimators = 10       # Chỉ train 10 cây cho lẹ
        n_splits = 2            # Chỉ chia 2 fold
    else:
        print("⚙️ PRODUCTION MODE (Chạy thật)")
        n_estimators = 300    
        n_splits = 3
        
        # Tự động phát hiện môi trường Kaggle
        if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') == 'Interactive':
            mode = 'validate'
            print("   -> Đang chạy Interactive: Chế độ VALIDATE")
        else:
            mode = 'submit'
            print("   -> Đang chạy Submit/Batch: Chế độ SUBMIT")

    # ====================================================

    cv = StratifiedGroupKFold(n_splits)


🚀 DETECTED SUBMISSION: Hệ thống tự động chuyển DEBUG = False để train full!
⚙️ PRODUCTION MODE (Chạy thật)
   -> Đang chạy Submit/Batch: Chế độ SUBMIT


In [7]:
def get_model_zoo():
    # choose sample size (very arbitrary number here)
    N_SAMPLES = 2_000_000 
    
    xgb_params = dict(
        verbosity=0, random_state=42,
        n_estimators=CFG.n_estimators, # Lấy từ CFG
        learning_rate=0.05, 
        max_depth=8,
        min_child_weight=3, 
        subsample=0.8, 
        colsample_bytree=0.8,
        scale_pos_weight=3.0,
        tree_method='gpu_hist',       # GPU!!!
        predictor='gpu_predictor'     
    )

    xgb_pipe = make_pipeline(
        StratifiedSubsetClassifier(
            estimator=XGBClassifier(**xgb_params),
            n_samples=N_SAMPLES
        )
    )

    cat_params = dict(
        verbose=0, 
        random_state=42,
        iterations=CFG.n_estimators,   # Lấy từ CFG
        learning_rate=0.05, 
        depth=8,
        auto_class_weights='Balanced',
        allow_writing_files=False,
        task_type="GPU",              # Dùng GPU
        devices='0'
    )
    cat_pipe = make_pipeline(
        StratifiedSubsetClassifier(
            estimator=CatBoostClassifier(**cat_params),
            n_samples=N_SAMPLES
        )
    )
    

    lgbm_params = dict(
        verbosity=-1, 
        random_state=42,
        n_estimators=CFG.n_estimators, # Lấy từ CFG
        learning_rate=0.05, 
        max_depth=8,
        class_weight='balanced',       
        subsample=0.8, 
        colsample_bytree=0.8
    )
    lgbm_pipe = make_pipeline(
        StratifiedSubsetClassifier(
            estimator=LGBMClassifier(**lgbm_params),
            n_samples=N_SAMPLES
        )
    )
    
    return [('xgb', xgb_pipe), ('cat', cat_pipe), ('lgbm', lgbm_pipe)]

# Data loading and preprocessing

In [8]:
train = pd.read_csv(CFG.train_path)
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~lab_id.str.startswith('MABe22_')")

test = pd.read_csv(CFG.test_path)

In [9]:
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))
print(body_parts_tracked_list)

['["body_center", "ear_left", "ear_right", "forepaw_left", "forepaw_right", "hindpaw_left", "hindpaw_right", "neck", "nose", "tail_base", "tail_midpoint", "tail_tip"]', '["body_center", "ear_left", "ear_right", "headpiece_bottombackleft", "headpiece_bottombackright", "headpiece_bottomfrontleft", "headpiece_bottomfrontright", "headpiece_topbackleft", "headpiece_topbackright", "headpiece_topfrontleft", "headpiece_topfrontright", "lateral_left", "lateral_right", "neck", "nose", "tail_base", "tail_midpoint", "tail_tip"]', '["body_center", "ear_left", "ear_right", "hip_left", "hip_right", "lateral_left", "lateral_right", "nose", "spine_1", "spine_2", "tail_base", "tail_middle_1", "tail_middle_2", "tail_tip"]', '["body_center", "ear_left", "ear_right", "lateral_left", "lateral_right", "neck", "nose", "tail_base", "tail_midpoint", "tail_tip"]', '["body_center", "ear_left", "ear_right", "lateral_left", "lateral_right", "nose", "tail_base", "tail_tip"]', '["body_center", "ear_left", "ear_right"

In [10]:
def create_solution_df(dataset):
    """
    tổng hợp label (ground truth) từ các video.
    """
    solution = []
    
    # Duyệt qua từng dòng (từng video) trong metadata
    # tqdm = progress bar
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    
        lab_id = row['lab_id']
        
        # ko có tracking data
        if lab_id.startswith('MABe22'): 
            continue
        
        video_id = row['video_id']
        
        # Tạo đường dẫn đến file annotation
        path = f"{CFG.train_annotation_path}/{lab_id}/{video_id}.parquet"
        
        # ko có thì bỏ qua (thực sự dính 1 file :v)
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            continue
    
        # Gán lại metadata vào dataframe vừa đọc
        annot['lab_id'] = lab_id
        annot['video_id'] = video_id
        annot['behaviors_labeled'] = row['behaviors_labeled'] # Các hành vi cần dự đoán
        
        # - Nếu là pair action: đổi thành "mouse{id}"
        # - Nếu là self action: đổi thành "self"
        annot['target_id'] = np.where(
            annot.target_id != annot.agent_id, 
            annot['target_id'].apply(lambda s: f"mouse{s}"), 
            'self'
        )
        
        # agent_id: đổi thành định dạng "mouse{id}" (ví dụ: mouse0, mouse1)
        annot['agent_id'] = annot['agent_id'].apply(lambda s: f"mouse{s}")
        
        solution.append(annot)
    
    solution = pd.concat(solution)
    
    return solution

def validate(mode):
    if(mode == 'validate'):
        solution = create_solution_df(train_without_mabe22)
# Chỉ chạy khi đang ở chế độ 'validate' (kiểm thử)
if CFG.mode == 'validate':
    # Tạo dataframe solution từ tập dữ liệu validation (đã loại bỏ MABe22)
    solution = create_solution_df(train_without_mabe22)

## Data generator

In [11]:
# drop rare body parts
drop_body_parts =  [
    'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]

In [12]:

def load_tracking(lab_id, video_id, base_path):
    path = f"{base_path}/{lab_id}/{video_id}.parquet"
    return pd.read_parquet(path), path

def clean_bodyparts(df):
    if len(np.unique(df.bodypart)) > 5:
        return df.query("~ bodypart.isin(@drop_body_parts)")
    return df

# pivot + scale pixel to cm
def normalize_tracking(df, pix_per_cm):
    pvid = df.pivot(
        columns=["mouse_id", "bodypart"],
        index="video_frame",
        values=["x", "y"]
    )
    pvid = (
        pvid.reorder_levels([1, 2, 0], axis=1)
        .T.sort_index().T
    )
    return pvid / pix_per_cm

#  Parse behaviors_labeled 
def parse_behaviors(behavior_str):
    behaviors = json.loads(behavior_str)
    behaviors = [b.replace("'", "") for b in behaviors]
    behaviors = [b.split(',') for b in behaviors]
    return pd.DataFrame(behaviors, columns=['agent', 'target', 'action'])

# Load annotation (train only)
def load_annotation(tracking_path):
    anno_path = tracking_path.replace('train_tracking', 'train_annotation')
    try:
        return pd.read_parquet(anno_path)
    except FileNotFoundError:
        return None

# Build single-mouse labels
def build_single_label(annot, agent_id, actions, index):
    y = pd.DataFrame(0.0, columns=actions, index=index)
    subset = annot.query(
        "(agent_id == @agent_id) & (target_id == @agent_id)"
    )
    for _, arow in subset.iterrows():
        y.loc[arow.start_frame:arow.stop_frame, arow.action] = 1.0
    return y

# 7. Build pair labels 
def build_pair_label(annot, agent_id, target_id, actions, index):
    y = pd.DataFrame(0.0, columns=actions, index=index)
    subset = annot.query(
        "(agent_id == @agent_id) & (target_id == @target_id)"
    )
    for _, arow in subset.iterrows():
        y.loc[arow.start_frame:arow.stop_frame, arow.action] = 1.0
    return y


In [13]:
# generate self features for a single video
def generate_single_samples(pvid, behaviors, annot, video_id, is_train):
    # get self action rows
    single_behaviors = behaviors.query("target == 'self'")
    agents = np.unique(single_behaviors.agent)

    # Iterate over each agent (mouse)
    for agent_str in agents:
        agent = int(agent_str[-1])
        actions = np.unique(single_behaviors.query(
            "agent == @agent_str"
        ).action)

        try:
            X = pvid.loc[:, agent]
        except KeyError:
            continue
        # Create Metadata for the current sample
        # critical for mapping predictions back to the original video, agent, and frame.
        meta = pd.DataFrame({
            "video_id": video_id,
            "agent_id": agent_str,
            "target_id": "self",
            "video_frame": X.index
        })

        if is_train:
            y = build_single_label(annot, agent, actions, X.index)
            yield "single", X, meta, y
        else:
            yield "single", X, meta, actions


In [14]:
# create pair features
import itertools

def generate_pair_samples(pvid, behaviors, annot, video_id, is_train):
    pair_beh = behaviors.query("target != 'self'")
    if len(pair_beh) == 0:
        return

    mice = np.unique(pvid.columns.get_level_values('mouse_id'))

    for agent, target in itertools.permutations(mice, 2):
        a_str, t_str = f"mouse{agent}", f"mouse{target}"
        actions = np.unique(pair_beh.query(
            "(agent == @a_str) & (target == @t_str)"
        ).action)

        X = pd.concat([pvid[agent], pvid[target]], axis=1, keys=["A", "B"])

        meta = pd.DataFrame({
            "video_id": video_id,
            "agent_id": a_str,
            "target_id": t_str,
            "video_frame": X.index
        })

        if is_train:
            y = build_pair_label(annot, agent, target, actions, X.index)
            yield "pair", X, meta, y
        else:
            yield "pair", X, meta, actions


In [15]:
def generate_mouse_data(dataset, traintest,
                        traintest_directory=None,
                        generate_single=True,
                        generate_pair=True):

    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    for _, row in dataset.iterrows():

        if row.lab_id.startswith("MABe22") or type(row.behaviors_labeled) != str:
            continue

        # Load tracking
        df, track_path = load_tracking(row.lab_id, row.video_id, traintest_directory)
        df = clean_bodyparts(df)
        pvid = normalize_tracking(df, row.pix_per_cm_approx)

        # interpolate to fill
        pvid = pvid.interpolate(method='linear', limit_direction='both', axis=0)
        
        # Savitzky-Golay filter (Giúp loại bỏ rung nhiễu)
        if len(pvid) > 7:
            try:
                # window_length=7 (tương đương 0.2s), polyorder=2
                pvid.iloc[:] = savgol_filter(pvid.values, window_length=7, polyorder=2, axis=0)
            except Exception:
                pass
        # Parse behaviors
        behaviors = parse_behaviors(row.behaviors_labeled)

        # Load labels (train)
        annot = None
        if traintest == "train":
            annot = load_annotation(track_path)
            if annot is None:
                continue

        # Generate samples
        if generate_single:
            yield from generate_single_samples(
                pvid, behaviors, annot, row.video_id, traintest == 'train'
            )

        if generate_pair:
            yield from generate_pair_samples(
                pvid, behaviors, annot, row.video_id, traintest == 'train'
            )


## Transforming coordinates

In [16]:
def safe_rolling(series, window, func, min_periods=None):
    if min_periods is None:
        min_periods = max(1, window // 4)
    return series.rolling(window, min_periods=min_periods, center=True).apply(func, raw=True)

def _scale(n_frames_at_30fps, fps, ref=30.0):
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

Tạo đặc trưng từ dữ liệu tọa độ chuỗi thời gian

In [17]:
def add_curvature_features(X, center_x, center_y, fps):
    """
    Tính toán các đặc trưng liên quan đến độ cong (curvature) và độ ngoặt (turning) của quỹ đạo.
    Giúp nhận biết các hành vi xoay vòng, đảo chiều hoặc di chuyển phức tạp.
    """
    # Vận tốc (Velocity): Đạo hàm bậc 1 của vị trí
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    # Gia tốc (Acceleration): Đạo hàm bậc 1 của vận tốc (bậc 2 của vị trí)
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    # Tính độ cong (Curvature)
    # Công thức độ cong k = |x'y'' - y'x''| / (x'^2 + y'^2)^(3/2)
    # Tích chéo (Cross product) giữa vector vận tốc và gia tốc
    cross_prod = vel_x * acc_y - vel_y * acc_x
    # Độ lớn vận tốc (tốc độ tức thời)
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    # Tính độ cong (thêm 1e-6 để tránh lỗi chia cho 0 khi đứng yên)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)

    # Tính trung bình độ cong trong các cửa sổ trượt (rolling windows) khác nhau
    # _scale(w, fps) là hàm quy đổi từ thời gian/frames sang kích thước cửa sổ
    for w in [25, 50, 75]:
        ws = _scale(w, fps)
        # min_periods giúp tính toán được ngay cả khi chưa đủ dữ liệu (ở đầu chuỗi)
        X[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 5)).mean()

    # Tính tốc độ ngoặt (Turn Rate)
    # Góc của vector vận tốc (hướng di chuyển)
    angle = np.arctan2(vel_y, vel_x)
    # Sự thay đổi góc giữa các frame liên tiếp (độ lớn góc ngoặt)
    angle_change = np.abs(angle.diff())
    
    # Tính tổng góc ngoặt trong một khoảng thời gian (đo lường độ "ngoằn ngoèo" của đường đi)
    w = 30
    ws = _scale(w, fps)
    X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 5)).sum()

    return X


def add_multiscale_features(X, center_x, center_y, fps):
    """
    Tính toán đặc trưng tốc độ ở nhiều quy mô (scale) thời gian khác nhau.
    Giúp phân biệt hành vi ngắn hạn (giật mình) vs dài hạn (di chuyển tuần tra).
    """
    # Tính tốc độ vô hướng (cm/s hoặc px/s tùy đơn vị gốc)
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    scales = [20, 40, 60, 80]
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            # Tốc độ trung bình trong cửa sổ
            X[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            # Độ biến động của tốc độ (Standard Deviation) - xem con vật di chuyển đều hay giật cục
            X[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).std()

    # Tỷ lệ giữa tốc độ ngắn hạn và dài hạn (ví dụ: đang tăng tốc đột ngột so với mức bình thường)
    if len(scales) >= 2 and f'sp_m{scales[0]}' in X.columns and f'sp_m{scales[-1]}' in X.columns:
        X['sp_ratio'] = X[f'sp_m{scales[0]}'] / (X[f'sp_m{scales[-1]}'] + 1e-6)

    return X


def add_state_features(X, center_x, center_y, fps):
    """
    Phân loại trạng thái di chuyển (Đứng yên, Đi chậm, Chạy...) và tính toán tần suất xuất hiện.
    """
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    
    # Làm mượt tốc độ bằng Moving Average trước khi phân loại để giảm nhiễu
    w_ma = _scale(15, fps)
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()

    try:
        # --- Phân chia trạng thái (Discretization) ---
        # Các ngưỡng (bins): <0.5 (Đứng yên), 0.5-2.0 (Chậm), 2.0-5.0 (Vừa), >5.0 (Nhanh)
        # Lưu ý: Các con số này phụ thuộc vào đơn vị của dữ liệu đầu vào (pixel hay cm)
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        # Gán nhãn trạng thái: 0, 1, 2, 3
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        for window in [20, 40, 60, 80]:
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                # Tính tỷ lệ thời gian ở trong từng trạng thái (ví dụ: dành 80% thời gian để đứng yên)
                for state in [0, 1, 2, 3]:
                    X[f's{state}_{window}'] = (
                        (speed_states == state).astype(float)
                        .rolling(ws, min_periods=max(1, ws // 5)).mean()
                    )
                
                # Tính số lần chuyển trạng thái (State transitions)
                # Đo lường sự ổn định của hành vi (có hay thay đổi tốc độ liên tục không)
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 5)).sum()
    except Exception:
        pass # Bỏ qua nếu có lỗi (ví dụ dữ liệu quá ngắn)

    return X


def add_longrange_features(X, center_x, center_y, fps):
    """
    Các đặc trưng dài hạn: So sánh vị trí/tốc độ hiện tại với lịch sử quá khứ xa hơn.
    """
    # 1. Vị trí trung bình trong cửa sổ lớn (Rolling Mean)
    for window in [30, 60, 120]:
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            X[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            X[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()

    # 2. Vị trí trung bình theo trọng số mũ (Exponential Weighted Mean - EWM)
    # EWM giúp mượt hóa quỹ đạo nhưng vẫn bám sát các thay đổi gần nhất hơn là Rolling Mean
    for span in [30, 60, 120]:
        s = _scale(span, fps)
        X[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()

    # 3. Xếp hạng phần trăm tốc độ (Percentile Rank)
    # Tốc độ hiện tại nằm ở mức nào so với quá khứ (ví dụ: cao hơn 90% thời gian trước đó)
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    for window in [30, 60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            X[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)

    return X



Add features

In [18]:

def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    """
    Quan trọng cho MABe Challenge: Tính tương tác xã hội giữa 2 con chuột (A và B).
    """
    # Kiểm tra xem dữ liệu body_center của cả 2 con có tồn tại không
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    # Vector khoảng cách từ B đến A
    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    # Vận tốc riêng của từng con
    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()

    # Cosine Similarity giữa Vector vận tốc và Vector khoảng cách
    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    # Lưu ý dấu trừ (-rel_x) vì vector khoảng cách tính từ B đến A ngược lại
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    for window in [30, 60]:
        ws = _scale(window, fps)
        # Trung bình mức độ hướng về nhau trong khoảng thời gian
        X[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        X[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()

    # there are 26 behaviors, we are gonna do 2?
    # approach: Tốc độ thu hẹp khoảng cách (- đạo hàm khoảng cách)
    approach = -rel_dist.diff()
    # chase: Kết hợp việc "đang lại gần" và "đang hướng mặt về phía đối phương"
    chase = approach * B_lead 
    
    w = 30
    ws = _scale(w, fps)
    X[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()

    # Speed Correlation
    for window in [60, 120]:
        ws = _scale(window, fps)
        A_sp = np.sqrt(A_vx**2 + A_vy**2)
        B_sp = np.sqrt(B_vx**2 + B_vy**2)
        X[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)

    return X


def add_egocentric_features(X, mouse_pair, avail_A, avail_B):
    """
    Xoay hệ trục tọa độ sao cho Chuột A nằm tại (0,0) và đầu hướng lên trên.
    Giúp mô hình biết chuột B đang ở vị trí nào tương đối so với hướng nhìn của A.
    """
    # Chỉ tính được khi có đủ Mũi và Đuôi để xác định hướng
    if all(p in avail_A for p in ['nose', 'tail_base']) and 'body_center' in avail_B:
        # Tọa độ A
        ax_tail = mouse_pair['A']['tail_base']['x']
        ay_tail = mouse_pair['A']['tail_base']['y']
        ax_nose = mouse_pair['A']['nose']['x']
        ay_nose = mouse_pair['A']['nose']['y']

        # Tọa độ B
        bx = mouse_pair['B']['body_center']['x']
        by = mouse_pair['B']['body_center']['y']

        # 1. Tịnh tiến: Đưa đuôi A về gốc (0,0)
        dx = bx - ax_tail
        dy = by - ay_tail

        # 2. Tính góc quay (để A hướng thẳng đứng lên trục Y)
        vx = ax_nose - ax_tail
        vy = ay_nose - ay_tail
        angle_a = np.arctan2(vy, vx)
        theta = np.pi/2 - angle_a # Góc cần xoay
        
        cos_t = np.cos(theta)
        sin_t = np.sin(theta)

        # 3. Xoay tọa độ của B
        X['ego_bx'] = dx * cos_t - dy * sin_t
        X['ego_by'] = dx * sin_t + dy * cos_t
        
        # 4. Xoay cả vận tốc của B (Quan trọng để biết B đang lao tới hay bỏ chạy)
        if 'body_center' in avail_B:
             b_vx = mouse_pair['B']['body_center']['x'].diff().fillna(0)
             b_vy = mouse_pair['B']['body_center']['y'].diff().fillna(0)
             X['ego_b_vx'] = b_vx * cos_t - b_vy * sin_t
             X['ego_b_vy'] = b_vx * sin_t + b_vy * cos_t

    return X

def add_behavior_specific_features(X, mouse_data, behavior_type, fps):
    """
    Thêm features đặc trưng riêng cho từng loại hành vi
    """
    
    if behavior_type == 'single':
        # REAR (Đứng dậy 2 chân sau)
        # Đặc trưng: Thân cao lên, đầu cao, tốc độ chậm
        if 'body_center' in mouse_data.columns.get_level_values(0):
            cx = mouse_data['body_center']['x']
            cy = mouse_data['body_center']['y']
            
            # Độ cao trung bình của body_center
            X['body_height_mean'] = cy.rolling(_scale(30, fps)).mean()
            X['body_height_std'] = cy.rolling(_scale(30, fps)).std()
            
            # Rear thường có tốc độ di chuyển X gần 0 (đứng yên tại chỗ)
            X['horizontal_stillness'] = (
                cx.diff().abs().rolling(_scale(20, fps)).mean()
            )
    
    elif behavior_type == 'pair':
        if 'body_center' not in mouse_data['A'].columns.get_level_values(0):
            return X
        
        # CHASE (Đuổi bắt)
        # Đặc trưng: A chạy nhanh VỀ PHÍA B, khoảng cách thu hẹp nhanh
        A_cx = mouse_data['A']['body_center']['x']
        A_cy = mouse_data['A']['body_center']['y']
        B_cx = mouse_data['B']['body_center']['x']
        B_cy = mouse_data['B']['body_center']['y']
        
        dist = np.sqrt((A_cx - B_cx)**2 + (A_cy - B_cy)**2)
        
        # Tốc độ thay đổi khoảng cách (càng âm = đang đuổi kịp)
        X['chase_closing_speed'] = -dist.diff() * fps
        
        # Chase có tính tuần hoàn: gần → xa → gần
        X['chase_periodicity'] = dist.rolling(_scale(60, fps)).apply(
            lambda x: len(np.where(np.diff(np.sign(np.diff(x))))[0])  # Đếm số lần đổi chiều
        )
        
        # === SUBMIT (Chịu thua) ===
        # Đặc trưng: Nằm bẹp, ít cử động, B ở trên/gần A
        A_speed = np.sqrt(A_cx.diff()**2 + A_cy.diff()**2)
        B_speed = np.sqrt(B_cx.diff()**2 + B_cy.diff()**2)
        
        # A gần như đứng yên
        X['submit_immobility'] = (A_speed < 0.5).astype(float).rolling(
            _scale(30, fps)
        ).mean()
        
        # B di chuyển nhiều hơn A (đang thống trị)
        X['submit_dominance_ratio'] = B_speed / (A_speed + 1e-6)
        
        # ESCAPE (Bỏ chạy)
        # Đặc trưng: A chạy rất nhanh RA XA B
        rel_x = A_cx - B_cx
        rel_y = A_cy - B_cy
        rel_dist = np.sqrt(rel_x**2 + rel_y**2)
        
        # Vector vận tốc A
        A_vx = A_cx.diff()
        A_vy = A_cy.diff()
        
        # Kiểm tra A có chạy THẲNG ra xa B không
        # (Cosine similarity giữa hướng chạy và vector xa B)
        escape_alignment = (A_vx * rel_x + A_vy * rel_y) / (
            np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6
        )
        
        # Escape: alignment dương + tốc độ cao
        X['escape_score'] = escape_alignment * A_speed
        X['escape_acceleration'] = A_speed.diff()  # Tăng tốc đột ngột
    
    return X

In [19]:
def add_context(X):
    # Thêm ngữ cảnh quá khứ/tương lai cho các biến quan trọng
    # Chọn ra vài biến quan trọng nhất để tạo lag (đỡ nặng RAM)
    cols_to_lag = []
    
    # Ưu tiên các biến tốc độ và khoảng cách
    for c in ['sp_m20', 'd_m30', 'angle_diff', 'head_area']: 
        if c in X.columns:
            cols_to_lag.append(c)

    for col in cols_to_lag:
        # Lấy thông tin 10 frame trước (khoảng 0.3s)
        X[f'{col}_prev10'] = X[col].shift(10).fillna(method='bfill')
        # Lấy thông tin 10 frame sau (Future context - rất mạnh cho Offline detection)
        X[f'{col}_next10'] = X[col].shift(-10).fillna(method='ffill')
        # Tính độ thay đổi (Delta)
        X[f'{col}_delta'] = X[col] - X[f'{col}_prev10']

In [20]:
def transform_single(single_mouse, body_parts_tracked, fps):
    available_body_parts = single_mouse.columns.get_level_values(0)

    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    })
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    # Tính diện tích tam giác đầu (Nose - EarL - EarR)
    # phát hiện hành vi REAR (đứng lên) hoặc cúi đầu
    if all(p in available_body_parts for p in ['nose', 'ear_left', 'ear_right']):
        x1, y1 = single_mouse['nose']['x'], single_mouse['nose']['y']
        x2, y2 = single_mouse['ear_left']['x'], single_mouse['ear_left']['y']
        x3, y3 = single_mouse['ear_right']['x'], single_mouse['ear_right']['y']
        
        X['head_area'] = 0.5 * np.abs(x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2))
        
        # Biến thiên diện tích (đang co lại hay to ra)
        X['head_area_change'] = X['head_area'].diff()
        
    
    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps)
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        speeds = pd.DataFrame({
            'sp_lf': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
            'sp_rt': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
            'sp_lf2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
            'sp_rt2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                   cy.diff().rolling(ws, min_periods=1).var())

        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)

    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)
        w = _scale(30, fps)
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    X = add_behavior_specific_features(X, single_mouse, 'single', fps)

    add_context(X)
    X = X.fillna(X.median())
    X = X.astype(np.float16, copy=False)
    X = X.replace([np.inf, -np.inf], np.nan)
    
    return X

In [21]:
def transform_pair(mouse_pair, body_parts_tracked, fps):
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    # Tính vector hướng cơ thể (Nose -> Tail base)
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        # Vector A
        vec_A_x = mouse_pair['A']['nose']['x'] - mouse_pair['A']['tail_base']['x']
        vec_A_y = mouse_pair['A']['nose']['y'] - mouse_pair['A']['tail_base']['y']
        # Vector B
        vec_B_x = mouse_pair['B']['nose']['x'] - mouse_pair['B']['tail_base']['x']
        vec_B_y = mouse_pair['B']['nose']['y'] - mouse_pair['B']['tail_base']['y']
        
        # Góc của từng con so với trục tọa độ
        ang_A = np.arctan2(vec_A_y, vec_A_x)
        ang_B = np.arctan2(vec_B_y, vec_B_x)
        
        # Góc tương đối giữa 2 con (từ 0 đến PI)
        # Giúp phân biệt: Đối đầu vs Đuổi theo vs Vuông góc
        X['angle_diff'] = np.abs(np.arctan2(np.sin(ang_A - ang_B), np.cos(ang_A - ang_B)))

    # Khoảng cách chéo: Mũi A đến Đuôi B (và ngược lại)
    if ('nose' in avail_A and 'tail_base' in avail_B):
        X['dist_noseA_tailB'] = np.sqrt(
            (mouse_pair['A']['nose']['x'] - mouse_pair['B']['tail_base']['x'])**2 + 
            (mouse_pair['A']['nose']['y'] - mouse_pair['B']['tail_base']['y'])**2
        )
        
    if ('nose' in avail_B and 'tail_base' in avail_A):
        X['dist_noseB_tailA'] = np.sqrt(
            (mouse_pair['B']['nose']['x'] - mouse_pair['A']['tail_base']['x'])**2 + 
            (mouse_pair['B']['nose']['y'] - mouse_pair['A']['tail_base']['y'])**2
        )
        
    #*********************************************
    
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past

    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                     (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls']   = ((cd >= 5.0) & (cd < 15.0)).astype(float)
        X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far']   = (cd >= 30.0).astype(float)

    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}']  = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}']  = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var)

            Axd = mouse_pair['A']['body_center']['x'].diff()
            Ayd = mouse_pair['A']['body_center']['y'].diff()
            Bxd = mouse_pair['B']['body_center']['x'].diff()
            Byd = mouse_pair['B']['body_center']['y'].diff()
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()

    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}']  = nn.shift(l)
            X[f'nn_ch{lag}']  = nn - nn.shift(l)
            is_cl = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}']  = is_cl.rolling(l, min_periods=1).mean()

    if 'body_center' in avail_A and 'body_center' in avail_B:
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        val = (Avx * Bvx + Avy * Bvy) / (np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for off in [-30, -20, -10, 0, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)

        w = _scale(30, fps)
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)

        X = add_interaction_features(X, mouse_pair, avail_A, avail_B, fps)

        X = add_egocentric_features(X, mouse_pair, avail_A, avail_B)

    X = add_behavior_specific_features(X, mouse_pair, 'pair', fps)

    
    add_context(X)
    X = X.fillna(X.median())
    X = X.astype(np.float16, copy=False)
    X = X.replace([np.inf, -np.inf], np.nan)

    return X

In [22]:
def robustify(submission, dataset, traintest, traintest_directory=None):
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"


    old_submission = submission.copy()
    # simple sanity check: start < stop
    submission = submission[submission.start_frame < submission.stop_frame]

    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")
    
    # An agent (Mouse A) cannot perform two different actions on the SAME Target (Mouse B) at the SAME time.
    old_submission = submission.copy()
    group_list = []

    # iterate by (video, agent, target)
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        # sort by time (start_frame)
        group = group.sort_values('start_frame')

        mask = np.ones(len(group), dtype=bool)  # giữ lại dòng không bị overlap
        last_stop_frame = 0

        for i, (_, row) in enumerate(group.iterrows()):
            # Nếu start < stop của đoạn trước → overlap → bỏ
            if row['start_frame'] < last_stop_frame:
                mask[i] = False
            else:
                # cập nhật last_stop_frame
                last_stop_frame = row['stop_frame']

        group_list.append(group[mask])
        
    submission = pd.concat(group_list)
    
    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")
        

    # ----------------------------- #
    # 3) Xử lý các video *không có bất kỳ prediction nào*
    #    Nhưng có nhãn trong dataset
    #    → Tạo các prediction dummy để tránh lỗi submission
    # ----------------------------- #
    s_list = []

    # Duyệt tất cả video trong dataset
    for idx, row in dataset.iterrows():
        lab_id = row['lab_id']

        # Bỏ qua video thuộc MABe22 (không cần robustify)
        if lab_id.startswith('MABe22'):
            continue
        
        video_id = row['video_id']

        # Nếu video này đã có prediction → bỏ qua
        if (submission.video_id == video_id).any():
            continue
        
        # Nếu hàng dataset này không có behaviors_labeled → bỏ qua
        if type(row.behaviors_labeled) != str:
            continue

        print(f"Video {video_id} has no predictions.")
        
        # Đọc file tracking
        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        # behaviors_labeled là list dạng chuỗi → parse & làm sạch
        vid_behaviors = json.loads(row['behaviors_labeled'])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
    
        # Tính frame bắt đầu & kết thúc toàn video
        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1
    
        # Gộp theo (agent, target)
        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            # Chia đều độ dài video theo số lượng action
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))

            # Tạo các đoạn dummy
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)

                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    # Nếu có dummy predictions → thêm vào submission
    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])
        print("ERROR: Filled empty videos")

    # Reset index sau khi concat
    submission = submission.reset_index(drop=True)
    
    return submission



In [23]:
def predict_multiclass(pred, meta, thresholds):
    # -------------------------------------------------------
    # pred: DataFrame (n_frames × n_actions) chứa xác suất từng action tại mỗi frame
    # meta: DataFrame chứa video_id, agent_id, target_id, video_frame
    # thresholds: dict quy định ngưỡng cho từng action (VD: {'attack':0.32,...})
    # Mục tiêu: Chuyển chuỗi dự đoán frame-by-frame → các segment hành vi
    # -------------------------------------------------------

    # use rolling mean for smooth prob
    # 5 frames
    pred = pred.rolling(window=5, min_periods=1, center=True).mean()

    # 1) Với mỗi frame, chọn action có xác suất cao nhất (argmax)
    ama = np.argmax(pred.values, axis=1)

    # Lấy giá trị xác suất cao nhất tại frame đó
    max_proba = pred.max(axis=1).values

    # 2) Gán ngưỡng cho từng action theo dict threshold
    # Nếu action không có trong dict → dùng ngưỡng mặc định = 0.27
    threshold_array = np.array([thresholds.get(col, 0.27) for col in pred.columns])

    # Lấy ngưỡng tương ứng với action argmax tại từng frame
    action_thresholds = threshold_array[ama]

    # 3) Những frame có xác suất < threshold → gán -1 (tức là "no action")
    ama = np.where(max_proba >= action_thresholds, ama, -1)

    # Chuyển thành Series với index = video_frame (giúp xác định start/stop)
    ama = pd.Series(ama, index=meta.video_frame)

    # -------------------------------------------------------
    # BƯỚC 4: TÌM CÁC FRAME CÓ SỰ THAY ĐỔI ACTION
    # -------------------------------------------------------

    # Mặt nạ: frame hiện tại khác frame trước → có thay đổi hành vi
    changes_mask = (ama != ama.shift(1)).values

    # Lấy ra chỉ các frame nơi hành vi thay đổi
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]

    # -------------------------------------------------------
    # BƯỚC 5: XÂY DỰNG SUBMISSION TỪ CÁC ĐIỂM THAY ĐỔI
    # -------------------------------------------------------

    # Loại bỏ các frame có action = -1 (no action)
    mask = ama_changes.values >= 0

    # Frame cuối cùng không thể làm start của segment → bỏ
    mask[-1] = False

    # Tạo DataFrame chứa các segment
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values], # truy xuất tên action: pred.columns[index_of_action]
        'start_frame': ama_changes.index[mask],   # start = frame tại thay đổi
        'stop_frame': ama_changes.index[1:][mask[:-1]]  # stop = frame thay đổi tiếp theo
    })

    # -------------------------------------------------------
    # BƯỚC 6: XỬ LÝ TRƯỜNG HỢP BỊ GỘP NHẦM segment KHÁC VIDEO
    # -------------------------------------------------------
    # Nếu hai điểm thay đổi nằm ở video/agent/target khác nhau
    # → Không thể dùng frame kế tiếp làm stop_frame
    # → Phải set stop_frame = frame cuối của video đó

    stop_video_id  = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id  = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values

    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]

        # Nếu khác video/agent/target → dừng segment tại frame cuối video
        if stop_video_id[i] != video_id or stop_agent_id[i] != agent_id or stop_target_id[i] != target_id:

            # Lấy frame cuối của video
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1

            # Ghi đè stop_frame
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame

    return submission_part

# Chọn action có xác suất lớn nhất ở mỗi frame (argmax).
# Áp dụng ngưỡng (threshold) riêng cho từng action để loại bỏ các frame có độ tin cậy thấp (gán thành -1, tức "no action").
# Tìm các điểm thay đổi hành vi theo thời gian, ví dụ: no action → attack → chase → ....
# Ghép các điểm thay đổi thành các segment:
# start_frame = thời điểm bắt đầu hành động mới
# stop_frame = thời điểm hành động tiếp theo xuất hiện
# Sửa lỗi khi frame thay đổi rơi sang video khác, đảm bảo stop_frame không bao giờ thuộc video khác.
# Xuất ra DataFrame chuẩn submission:
# video_id, agent_id, target_id, action, start_frame, stop_frame.

In [24]:

def tune_threshold(oof_action, y_action):
    # simple linear scan
    thresholds = np.arange(0.1, 0.9, 0.02) 
    best_threshold = 0.5
    best_score = -1
    
    for th in thresholds:
        score = f1_score(y_action, (oof_action >= th), zero_division=0)
        if score > best_score:
            best_score = score
            best_threshold = th
            
    return best_threshold

In [25]:
def cross_validate_classifier(X, label, meta, body_parts_tracked_str, section):
    oof = pd.DataFrame(index=meta.video_frame)
    f1_list = []
    submission_list = []
    thresholds = {}
    
    # Lấy danh sách 3 mô hình
    models = get_model_zoo() 
    
    for action in label.columns:
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)
        X_action = X[action_mask]
        groups_action = meta.video_id[action_mask]
        
        if len(np.unique(groups_action)) < CFG.n_splits:
            continue

        if not (y_action == 0).all():
            try:
                # Biến chứa tổng xác suất của cả 3 mô hình
                ensemble_oof_preds = np.zeros(len(y_action))
                
                # Duyệt qua từng mô hình (xgb, cat, lgbm)
                for name, model_template in models:
                    with warnings.catch_warnings():
                        warnings.filterwarnings('ignore')
                        
                        # Đường dẫn lưu riêng cho từng model
                        # VD: xgboost/1/attack, catboost/1/attack...
                        save_path = f"{CFG.model_name}/{name}/{section}/{action}"
                        
                        trainer = Trainer(
                            estimator=clone(model_template),
                            cv=CFG.cv,
                            cv_args={"groups": groups_action},
                            metric=f1_score,
                            task="binary",
                            verbose=False,
                            save=True,
                            save_path=save_path
                        )

                        trainer.fit(X_action, y_action)
                        
                        # Cộng dồn kết quả (Chia trung bình sau)
                        ensemble_oof_preds += trainer.oof_preds
                        
                        # Lưu file OOF lẻ (nếu cần debug)
                        joblib.dump(trainer.oof_preds, f"{save_path}/oof.pkl")
                        del trainer
                        gc.collect()
                
                # Lấy trung bình cộng (Soft Voting)
                ensemble_oof_preds /= len(models) 

                # --- TỐI ƯU NGƯỠNG TRÊN KẾT QUẢ TỔNG HỢP ---
                threshold = tune_threshold(ensemble_oof_preds, y_action)
                thresholds[action] = threshold
        
                f1 = f1_score(y_action, (ensemble_oof_preds >= threshold), zero_division=0)
                f1_list.append((body_parts_tracked_str, action, f1))
                
                print(f"\t[Ensemble] F1: {f1:.4f} ({threshold:.2f}) - {action}")

            except Exception as e:
                # In lỗi chi tiết để debug
                print(f"\t!!! Error {action}: {e}")
                import traceback
                traceback.print_exc()
                oof_action = np.zeros(len(y_action))
        
        else:
            ensemble_oof_preds = np.zeros(len(y_action))
        
        # Lưu kết quả ensemble vào bảng OOF tổng
        oof_column = np.zeros(len(label))
        oof_column[action_mask] = ensemble_oof_preds
        oof[action] = oof_column

        gc.collect()

    submission_part = predict_multiclass(oof, meta, thresholds)
    submission_list.append(submission_part)
    
    return submission_list, f1_list, thresholds

In [26]:
# Thêm feature_map
def submit(body_parts_tracked_str, switch_tr, section, thresholds, feature_map=None):    
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
        
    test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
    
    fps_lookup = (
        test_subset[['video_id', 'frames_per_second']]
        .drop_duplicates('video_id')
        .set_index('video_id')['frames_per_second']
        .to_dict()
    )

    model_names = ['xgb', 'cat', 'lgbm']
    submission_list = []
    
    generator = generate_mouse_data(
        test_subset, 'test',
        generate_single=(switch_tr == 'single'), 
        generate_pair=(switch_tr == 'pair')
    )

    for switch_te, data_te, meta_te, actions_te in generator:
        try:
            fps_i = _fps_from_meta(meta_te, fps_lookup)
            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps_i)
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps_i)
            
            # FIX  FEATURE MISMATCH 
            if feature_map is not None:
                # Lấy danh sách cột từ file đã lưu lúc train
                # Key: "1_single" hoặc "1_pair"
                key = f"{section}_{switch_tr}"
                required_cols = feature_map.get(key)
                
                if required_cols is not None:
                    # bỏ cột thừa ở Test mà Train không có
                    # thêm cột thiếu và fill 0
                    # sắp xếp đúng thứ tự
                    X_te = X_te.reindex(columns=required_cols, fill_value=0)
            
            del data_te
            gc.collect()
    
            pred = pd.DataFrame(index=meta_te.video_frame)
            
            for action in actions_te:
                ensemble_prob = np.zeros(len(X_te))
                valid_models_count = 0
                
                for name in model_names:
                    path_pattern = f"{CFG.model_name}/{name}/{section}/{action}/*_trainer_*.pkl"
                    files = glob.glob(path_pattern)
                    
                    if len(files) >= 1:
                        trainer = joblib.load(files[0])
                        ensemble_prob += trainer.predict(X_te)
                        valid_models_count += 1
                        del trainer
                
                if valid_models_count > 0:
                    pred[action] = ensemble_prob / valid_models_count
                else:
                    pred[action] = 0.0
                
            del X_te
            gc.collect()

            if pred.shape[1] != 0:
                submission_part = predict_multiclass(pred, meta_te, thresholds)
                submission_list.append(submission_part)
                
        except KeyError:
            del data_te
            gc.collect()
            
    return submission_list

In [27]:
def smooth_predictions(submission, min_duration_frames=2, merge_gap_frames=3):
    """
    Lọc bỏ các segment quá ngắn và gộp các segment gần nhau
        min_duration_frames: Bỏ segment ngắn hơn N frames ()
        merge_gap_frames: Gộp nếu khoảng cách < N frames
    """
    result = []
    
    for (video, agent, target, action), group in submission.groupby(
        ['video_id', 'agent_id', 'target_id', 'action']
    ):
        group = group.sort_values('start_frame')
        
        segments = []
        for _, row in group.iterrows():
            duration = row['stop_frame'] - row['start_frame']
            
            #skip segment quá ngắn
            if duration < min_duration_frames:
                continue
            
            # merge vào segment trước nếu gần
            if segments and (row['start_frame'] - segments[-1][1]) <= merge_gap_frames:
                segments[-1] = (segments[-1][0], row['stop_frame'])
            else:
                segments.append((row['start_frame'], row['stop_frame']))
        
        for start, stop in segments:
            result.append({
                'video_id': video, 'agent_id': agent,
                'target_id': target, 'action': action,
                'start_frame': start, 'stop_frame': stop
            })
    
    return pd.DataFrame(result)

In [28]:
def run_validation(dataset, body_parts_tracked_list):
    print(f" running validation")
    
    f1_list = []
    submission_list = []
    learned_thresholds = {"single": {}, "pair": {}}
    # store features used for training
    feature_map = {} 

    for section in range(1, len(body_parts_tracked_list)):
        body_parts_tracked_str = body_parts_tracked_list[section]
        try:
            body_parts = json.loads(body_parts_tracked_str)
            print(f">>> Processing Section {section}/{len(body_parts_tracked_list)-1}")
            
            if len(body_parts) > 5:
                body_parts_tracked_cleaned = [b for b in body_parts if b not in drop_body_parts]
            
            subset = dataset[dataset.body_parts_tracked == body_parts_tracked_str]
            
            _fps_lookup = (
                subset[['video_id', 'frames_per_second']]
                .drop_duplicates('video_id')
                .set_index('video_id')['frames_per_second']
                .to_dict()
            )
            
            # --- Gom dữ liệu Single & Pair ---
            single_data, single_meta, single_label = [], [], []
            pair_data, pair_meta, pair_label = [], [], []
            
            for switch, data, meta, label in generate_mouse_data(subset, 'train'):
                if switch == 'single':
                    single_data.append(data); single_meta.append(meta); single_label.append(label)
                else:
                    pair_data.append(data); pair_meta.append(meta); pair_label.append(label)
                del data, meta, label
            gc.collect()

            # --- A. XỬ LÝ SINGLE MOUSE (CÓ CẮT GIẢM DỮ LIỆU) ---
            if len(single_data) > 0:
                print(f"   [Single Mouse] Training...")
                X_list, y_list, meta_list = [], [], []
                
                # Duyệt qua từng video
                for d, m, l in zip(single_data, single_meta, single_label):
                    fps = _fps_from_meta(m, _fps_lookup)
                    # Tạo feature
                    feat = transform_single(d, body_parts_tracked_cleaned, fps)
                    
                    # === CẮT NHỎ DỮ LIỆU TẠI ĐÂY (Downsampling) ===
                    # Chỉ lấy mỗi 2 dòng 1 dòng (Step = 2)
                    indices = np.arange(0, len(feat), 2)
                    
                    X_list.append(feat.iloc[indices])
                    y_list.append(l.iloc[indices])
                    meta_list.append(m.iloc[indices])
                
                # Nối lại (Lúc này dữ liệu đã nhỏ, không sợ sập RAM)
                X_tr = pd.concat(X_list, ignore_index=True).astype(np.float32)
                X_tr = X_tr.replace([np.inf, -np.inf], np.nan)
                
                y_tr = pd.concat(y_list, ignore_index=True)
                meta_tr = pd.concat(meta_list, ignore_index=True)
                
                # Xóa list cũ
                del single_data, single_meta, single_label, X_list, y_list, meta_list
                gc.collect()
                
                # Train
                # === [SỬA 2] LƯU DANH SÁCH CỘT SINGLE ===
                feature_map[f"{section}_single"] = X_tr.columns.tolist()
                # ========================================

                subs, f1s, threshs = cross_validate_classifier(X_tr, y_tr, meta_tr, body_parts_tracked_str, section)
                
                f1_list.extend(f1s)
                submission_list.extend(subs)
                learned_thresholds["single"][str(section)] = threshs
                
                del X_tr, y_tr, meta_tr
                gc.collect()

            # --- B. XỬ LÝ PAIR MOUSE (CÓ CẮT GIẢM DỮ LIỆU) ---
            if len(pair_data) > 0:
                print(f"   [Pair Mouse] Training...")
                X_list, y_list, meta_list = [], [], []
                
                for d, m, l in zip(pair_data, pair_meta, pair_label):
                    fps = _fps_from_meta(m, _fps_lookup)
                    feat = transform_pair(d, body_parts_tracked_cleaned, fps)
                    
                    # === CẮT NHỎ DỮ LIỆU TẠI ĐÂY ===
                    indices = np.arange(0, len(feat), 5)
                    
                    X_list.append(feat.iloc[indices])
                    y_list.append(l.iloc[indices])
                    meta_list.append(m.iloc[indices])
                
                X_tr = pd.concat(X_list, ignore_index=True).astype(np.float32)
                X_tr = X_tr.replace([np.inf, -np.inf], np.nan)
                y_tr = pd.concat(y_list, ignore_index=True)
                meta_tr = pd.concat(meta_list, ignore_index=True)
                
                del pair_data, pair_meta, pair_label, X_list, y_list, meta_list
                gc.collect()
                
                feature_map[f"{section}_pair"] = X_tr.columns.tolist()
                # ======================================

                subs, f1s, threshs = cross_validate_classifier(X_tr, y_tr, meta_tr, body_parts_tracked_str, section)
                
                f1_list.extend(f1s)
                submission_list.extend(subs)
                learned_thresholds["pair"][str(section)] = threshs
                
                del X_tr, y_tr, meta_tr
                gc.collect()
                
        except Exception as e:
            print(f"!!! LỖI tại Section {section}: {e}")
            import traceback
            traceback.print_exc()
            
    if f1_list:
        f1_df = pd.DataFrame(f1_list, columns=['config', 'action', 'f1'])
        if CFG.mode == 'validate':
            print("\n=== KẾT QUẢ VALIDATION ===")
            print(f"Mean F1 Score: {f1_df['f1'].mean():.4f}")
        
        joblib.dump(learned_thresholds, f"{CFG.model_name}_thresholds.pkl")
        print("Đã lưu ngưỡng học được vào file pkl")

        joblib.dump(feature_map, f"{CFG.model_name}_features.pkl")
        print("Đã lưu danh sách features vào file pkl")
        
    return f1_list

# Submission

In [29]:
def run_submission(dataset, body_parts_tracked_list):
    print(f"\n{'='*40}")
    print(f"  BẮT ĐẦU QUÁ TRÌNH: SUBMISSION")
    print(f"{'='*40}\n")
    
    submission_list = []
    
    try:
        loaded_thresholds = joblib.load(f"{CFG.model_name}_thresholds.pkl")
        loaded_features = joblib.load(f"{CFG.model_name}_features.pkl")
        print("Đã load thành công ngưỡng threshold và feature map từ file!")
    except:
        print("Không tìm thấy file pkl. Sẽ dùng mặc định (có thể gây lỗi lệch cột).")
        loaded_thresholds = {"single": {}, "pair": {}}
        loaded_features = {} 

    for section in range(1, len(body_parts_tracked_list)):
        body_parts_tracked_str = body_parts_tracked_list[section]
        try:
            print(f">>> Predicting Section {section}/{len(body_parts_tracked_list)-1}")
            
            thresh_single = loaded_thresholds["single"].get(str(section), {})
            thresh_pair = loaded_thresholds["pair"].get(str(section), {})
            
            subs_single = submit(body_parts_tracked_str, 'single', section, thresh_single, loaded_features)
            submission_list.extend(subs_single)
            
            subs_pair = submit(body_parts_tracked_str, 'pair', section, thresh_pair, loaded_features)
            submission_list.extend(subs_pair)
            
            gc.collect()
            
        except Exception as e:
            print(f"!!! LỖI tại Section {section}: {e}")
            import traceback
            traceback.print_exc()

    if len(submission_list) > 0:
        full_submission = pd.concat(submission_list)
        full_submission = smooth_predictions(
            full_submission,
            min_duration_frames=2,
            merge_gap_frames=3
        )
        final_submission = robustify(full_submission, dataset, 'test')
        
        final_submission.index.name = 'row_id'
        final_submission.to_csv('submission.csv')
        print("\nSUCCESS: Đã tạo file 'submission.csv'.")
        display(final_submission.head())
    else:
        print("\nWARNING: Không dự đoán được gì. Tạo file dummy.")
        # dummy fallback
        pd.DataFrame({'video_id': [438887472], 'action': ['rear']}).to_csv('submission.csv')

In [30]:

run_validation(train, body_parts_tracked_list)

if CFG.mode == 'submit' or CFG.debug:

    run_submission(test, body_parts_tracked_list)
else:
    print("validating.")

 running validation
>>> Processing Section 1/9
   [Single Mouse] Training...
	[Ensemble] F1: 0.4991 (0.50) - rear
   [Pair Mouse] Training...
	[Ensemble] F1: 0.4190 (0.40) - approach
	[Ensemble] F1: 0.4166 (0.20) - attack
	[Ensemble] F1: 0.4890 (0.60) - avoid
	[Ensemble] F1: 0.4981 (0.34) - chase
	[Ensemble] F1: 0.5354 (0.52) - chaseattack
	[Ensemble] F1: 0.0000 (0.10) - submit
>>> Processing Section 2/9
   [Single Mouse] Training...
	[Ensemble] F1: 0.6644 (0.38) - huddle
   [Pair Mouse] Training...
	[Ensemble] F1: 0.6233 (0.58) - reciprocalsniff
	[Ensemble] F1: 0.5385 (0.70) - sniffgenital
>>> Processing Section 3/9
   [Single Mouse] Training...
	[Ensemble] F1: 0.4263 (0.56) - rear
   [Pair Mouse] Training...
	[Ensemble] F1: 0.3164 (0.38) - approach
	[Ensemble] F1: 0.1299 (0.56) - attack
	[Ensemble] F1: 0.2014 (0.62) - avoid
	[Ensemble] F1: 0.0232 (0.48) - chase
	[Ensemble] F1: 0.1983 (0.32) - chaseattack
	[Ensemble] F1: 0.0968 (0.12) - submit
>>> Processing Section 4/9
   [Pair Mouse

,video_id,agent_id,target_id,action,start_frame,stop_frame
row_id,,,,,,
0,438887472,mouse1,mouse3,approach,2283,2292
1,438887472,mouse1,mouse4,attack,1272,1281
2,438887472,mouse1,mouse4,attack,1429,1433
3,438887472,mouse1,mouse4,attack,1453,1491
4,438887472,mouse1,mouse4,attack,1496,1527


In [31]:
result = pd.read_csv('submission.csv')
display(result.head())

,row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,0,438887472,mouse1,mouse3,approach,2283,2292
1,1,438887472,mouse1,mouse4,attack,1272,1281
2,2,438887472,mouse1,mouse4,attack,1429,1433
3,3,438887472,mouse1,mouse4,attack,1453,1491
4,4,438887472,mouse1,mouse4,attack,1496,1527
